In [2]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import janitor
import opendatasets as od
import os
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report
from sklearn.tree import DecisionTreeClassifier

### 1. Import Data

In [3]:
# pima_df = od.download("https://www.kaggle.com/datasets/kumargh/pimaindiansdiabetescsv", data_dir="./data") ##already executed.

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading pimaindiansdiabetescsv.zip to ./data/pimaindiansdiabetescsv


100%|██████████| 8.89k/8.89k [00:00<00:00, 2.85MB/s]

In [11]:
pima_df = pd.read_csv("./data/pima/pima-indians-diabetes.csv", header=0, sep=",")
pima_df.head()

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [17]:
pima_df.columns = ["pregnancies", "glucose", "blood_pressure", "skin_thickness", "insulin", "bmi", "pedigree", "age", "class"]

In [18]:
pima_df.head(2)

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,pedigree,age,class
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1


### 2. EDA

In [19]:
pima_df.sample(10)

,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,pedigree,age,class
326,10,179,70,0,0,35.1,0.200,37,0
5,3,78,50,32,88,31.0,0.248,26,1
640,4,128,70,0,0,34.3,0.303,24,0
757,1,106,76,0,0,37.5,0.197,26,0
46,2,71,70,27,0,28.0,0.586,22,0
342,5,122,86,0,0,34.7,0.290,33,0
204,5,111,72,28,0,23.9,0.407,27,0
233,3,74,68,28,45,29.7,0.293,23,0
447,0,104,64,37,64,33.6,0.510,22,1
380,0,105,68,22,0,20.0,0.236,22,0


In [20]:
pima_df.isna().sum()

pregnancies       0
glucose           0
blood_pressure    0
skin_thickness    0
insulin           0
bmi               0
pedigree          0
age               0
class             0
dtype: int64